## Векторизация данных

Для обработки будут использоваться GPU мощности Google Colab.

In [ ]:
import torch
import chromadb
import numpy as np
import pandas as pd
import torch.nn.functional as F

from tqdm import tqdm
from google.colab import drive
from chromadb.utils import embedding_functions
from transformers import AutoTokenizer, AutoModel

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


Подключаем GPU.

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"GPU detected: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("GPU not detected, using CPU")

GPU detected: Tesla T4


Будем использовать `USER-bge-m3`, источник: https://huggingface.co/deepvk/USER-bge-m3

**USER-bge-m3** — это универсальная векторная модель (Universal Sentence Encoder Representation), разработанная для представления текстов в виде векторов, оптимизированных для задач семантического поиска и сравнения текстов. Основные характеристики:

- Назначение:
  - Модель предназначена для создания компактных и информативных векторныхпредставлений предложений, текстов и документов, что делает ее полезной для:
    - Семантического поиска,
    - Сравнения текстов (поиск похожих),
    - Рекомендательных систем,
    - Кластеризации и классификации текстов.

- Архитектура:
  - Основана на трансформерах (Transformer), что обеспечивает высокую точность в понимании смысла текста.
  - Может быть оптимизирована под различные языки, включая русский.

- Поддержка API и интеграция:
  - Подключается через API Hugging Face и может быть использована в связке с библиотеками для обработки данных (например, FAISS, Chroma DB).

- Преимущества:
  - Универсальность: подходит для множества задач.
  - Высокая производительность при сравнении семантики текстов.

Определим параметры для базы данных.

In [ ]:
CHROMA_DATA_PATH = "/content/drive/MyDrive/ds/chroma/"
EMBED_MODEL = "deepvk/USER-bge-m3"
COLLECTION_NAME = "reviews_collection_02"

Инициализируем клиент БД.

In [ ]:
client = chromadb.PersistentClient(path=CHROMA_DATA_PATH)

Инициализируем модель и переключаем в режим оценки.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(EMBED_MODEL)
model = AutoModel.from_pretrained(EMBED_MODEL).to(device)
model.eval()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.33M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/963 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/697 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.44G [00:00<?, ?B/s]

XLMRobertaModel(
  (embeddings): XLMRobertaEmbeddings(
    (word_embeddings): Embedding(46166, 1024, padding_idx=1)
    (position_embeddings): Embedding(8194, 1024, padding_idx=1)
    (token_type_embeddings): Embedding(1, 1024)
    (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): XLMRobertaEncoder(
    (layer): ModuleList(
      (0-23): 24 x XLMRobertaLayer(
        (attention): XLMRobertaAttention(
          (self): XLMRobertaSdpaSelfAttention(
            (query): Linear(in_features=1024, out_features=1024, bias=True)
            (key): Linear(in_features=1024, out_features=1024, bias=True)
            (value): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): XLMRobertaSelfOutput(
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (LayerNorm): LayerNorm((1024,), eps=1e-05, eleme

Инициализируем функцию генерации эмбеддингов.

In [ ]:
embedding_func = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name=EMBED_MODEL,
    device="cuda" if torch.cuda.is_available() else "cpu"
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/195 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/9.34k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

Создаём коллекцию в БД.

In [ ]:
collection = client.create_collection(
    name=COLLECTION_NAME,
    embedding_function=embedding_func,
    metadata={"hnsw:space": "cosine"}
)

Загружаем подготовленный датасет.

In [ ]:
reviews = pd.read_csv('/content/drive/MyDrive/ds/v1.csv', sep=',')

print("Dataset Info:")
reviews.info()

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47769 entries, 0 to 47768
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   place_id        47769 non-null  int64  
 1   name_primary    47769 non-null  object 
 2   name_extension  47769 non-null  object 
 3   address         47769 non-null  object 
 4   lat             47769 non-null  float64
 5   lon             47769 non-null  float64
 6   rating_decimal  47769 non-null  float64
 7   avg_price       26704 non-null  float64
 8   review          47769 non-null  object 
 9   rating          47769 non-null  float64
 10  id              47769 non-null  int64  
dtypes: float64(5), int64(2), object(4)
memory usage: 4.0+ MB


Выделяем столбец, который будем векторизовать.

In [ ]:
documents = reviews['review'].values
documents[10]

'шальное заведение! находится в опасной близости от пирогов и пропаганды. всегда у нас с друзьями были планы вот только выпьем по шоту в бурбоне и плясать в другое место! в итоге каждый раз зависали там до 4 утра.... . мало места, накурено, крошечная сцена, к бару не протолкнуться, туалет с дурацкими дверьми аля салун но атмосфера безудержного веселья никуда не девается. не стоит надеяться найти там друзей, приходите со своими музыка уж больно хороша, отплясывать. а то к утра народ тихонечко напивается и начинает праздно шататься, и вот тогда лучше быть со своими.. . шоты в районе 270, пиво 250, бехеровка льется рекой, коктейли ок, но шоты лучше . . для разогрева место самое то! только раньше 23.00 появляться там не стоит слишком тихо.'

Инициализируем функцию для извлечения метаданных и извлекаем их.

In [ ]:
def extract_metadata(row):
    return {
        'name': row['name_primary'],
        "name_extension": row.get('name_extension', None),
        'address': row['address'],
        'lat': row['lat'],
        'lon': row['lon'],
        'rating': row['rating_decimal'],
        "avg_price": row.get('avg_price', None),
        'id': row['place_id']
    }

In [ ]:
metadata = [extract_metadata(row) for _, row in reviews.iterrows()]
metadata[0]

{'name': '495',
 'name_extension': 'пивоваренный ресторан',
 'address': 'Олимпийский проспект, 18/1',
 'lat': 55.785009,
 'lon': 37.624123,
 'rating': 5.0,
 'avg_price': 1500.0,
 'id': 4504127908348140}

Контроль размерности.

In [ ]:
print(len(documents))
print(len(metadata))

47769
47769


Запускаем обработку.

In [ ]:
batch_size = 1000

for i in tqdm(range(0, len(documents), batch_size), desc="Добавляем документы в коллекцию:"):
    batch_documents = documents[i:i + batch_size]
    batch_ids = [f"id{j}" for j in range(i, i + len(batch_documents))]
    collection.add(
        documents=list(batch_documents),
        ids=batch_ids,
        metadatas=list(metadata[i:i + len(batch_documents)])
    )

Добавляем документы в коллекцию:: 100%|██████████| 48/48 [27:45<00:00, 34.70s/it]


Проверим результат.

In [ ]:
collection.query(
    query_texts=["лучшая пицца"],
    n_results=3,
)

{'ids': [['id38795', 'id39343', 'id39333']],
 'embeddings': None,
 'documents': [['вкусная итальянская пицца',
   'самая вкусная, потому самая любимая пицца уже много лет.',
   'хорошая пицца по доступным ценам']],
 'uris': None,
 'data': None,
 'metadatas': [[{'address': 'проспект Мира, 119с520',
    'id': 70000001091774449,
    'lat': 55.827146,
    'lon': 37.631146,
    'name': 'траттория пицца пеппе',
    'name_extension': 'траттория',
    'rating': 5.0},
   {'address': 'проспект Будённого, 26к1',
    'id': 70000001091774664,
    'lat': 55.764141,
    'lon': 37.731182,
    'name': 'пипони',
    'name_extension': 'ресторан',
    'rating': 5.0},
   {'address': 'Троицкий административный округ, Краснопахорский район, деревня Красная Пахра, 144Б/2с3',
    'id': 70000001091774660,
    'lat': 55.448352,
    'lon': 37.290074,
    'name': 'pizza express 24',
    'name_extension': 'ресторан',
    'rating': 5.0}]],
 'distances': [[0.1777600646018982, 0.21652334928512573, 0.22318041324615479]

Таким образом мы сформировали базу данных, где отзывы представлены в виде векторов. Это даёт нам возможность обрабатывать запросы пользователей, искать соответствие и выдавать наиболее подходящий результат.